In [1]:
!nvidia-smi

Sat Jan  9 14:55:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install bert-for-tf2 >> /dev/null

In [3]:
import os
import sys
import re
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from  tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer

In [4]:
pip install tape_proteins >> /dev/null

ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.4.0 which is incompatible.
ERROR: botocore 1.19.51 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cd "/content/drive/My Drive/Colab Notebooks NU"

/content/drive/.shortcut-targets-by-id/1FqifPUlm6cgaaWF0H2AQqipYJL1XcCct/Colab Notebooks


In [7]:
import bert_thesis_experiments.utils as utils
import bert_thesis_experiments.my_models as my_models

In [118]:
#@title Choose and Load Model { vertical-output: true }
model = "bert_thesis_experiments/dl_models/bert-768-ConDo_fcnn_flatten_stp30k_voc30k_epoch5_bsz16" #@param ["bert_thesis_experiments/dl_models/bert-512-dense-droput_ls", "bert_thesis_experiments/dl_models/bert-512-bilstm", "bert_thesis_experiments/dl_models/bert-512-dense-droput_ls_stp10k_voc80k_trh05", "bert_thesis_experiments/dl_models/bert-512-bilstm_ly1_lr2e5_uniref_stp10k_voc80k", "bert_thesis_experiments/dl_models/bert-512-bilstm_stacked_lr2e5_uniref_stp10k_voc80k", "bert_thesis_experiments/dl_models/bert-768-stacked_bilstm_stp30k_voc30k_epoch3_bsz32", "bert_thesis_experiments/dl_models/bert-768-dense-droput_ls_stp30k_voc30k_trh05_epoch5_bsz32", "bert_thesis_experiments/dl_models/bert-768-ConDo_fcnn_flatten_stp30k_voc30k_epoch5_bsz16", "bert_thesis_experiments/dl_models/bert-768-classifier_fltn_dense200_stp30k_voc30k_epoch5_bsz32"]
model=load_model(model)

In [188]:
#@title Test Files name
seqfile = "DeepDom_Code/DeepDom-master/test_data/casp9_2domain_singleline_processed.fasta" #@param ["DeepDom_Code/DeepDom-master/test_data/casp9_2domain_singleline_processed.fasta", "DeepDom_Code/DeepDom-master/test_data/DCD_casp8_processed.fasta", "DeepDom_Code/DeepDom-master/test_data/FM_casp9_processed.fasta"]
labelfile = "DeepDom_Code/DeepDom-master/test_data/casp9_2domain_singleline_processed_labels.fasta" #@param ["DeepDom_Code/DeepDom-master/test_data/casp9_2domain_singleline_processed_labels.fasta", "DeepDom_Code/DeepDom-master/test_data/DCD_casp8_processed_labels.fasta", "DeepDom_Code/DeepDom-master/test_data/FM_casp9_processed_labels.fasta"]
label_file_unprocessed = "DeepDom_Code/DeepDom-master/test_data/casp9_2domain_singleline_labels.fasta" #@param ["DeepDom_Code/DeepDom-master/test_data/casp9_2domain_singleline_labels.fasta", "DeepDom_Code/DeepDom-master/test_data/DCD_casp8_labels.fasta", "DeepDom_Code/DeepDom-master/test_data/FM_casp9_labels.fasta"]
# seqfile = #@test_file_seq #file name of the processed sequence data (output from dataprocess.pl)
# labelfile= ''; #file name of the processed label data (output from dataprocess.pl)

In [189]:
(testids,testseqs) = utils.process_inputseqs(seqfile)
(test_label_ids, test_label_seqs) = utils.process_inputlabels(labelfile)

In [190]:
i = 0
print(len(testids), len(testseqs))
print(len(testseqs[i]), testseqs[i])
print(testids == test_label_ids)
print(len(testseqs) == len(test_label_seqs))

36 36
200 MSASKEIKSFLWTQSLRRELSGYCSNIKLQVVKDAQALLHGLDFSEVSNVQRLMRKERRDDNDLKRLRDLNQAVNNLVELKSTQQKSILRVGTLTSDDLLILAADLEKLKSKVIRTERPLSAGVYMGNLSSQQLDQRRALLNMIGMSGGNQGARAGRDGVVRVWDVKNAELLNNQFGTMPSLTLACLTKQGQVDLNDAVQ
True
True


In [12]:
# tokenizer = FullTokenizer(vocab_file=os.path.join("bert_thesis_experiments/working/", "protein_seq_words-vocab.txt"))
tokenizer = FullTokenizer(vocab_file=os.path.join("bert_thesis_experiments/working/", "protein_seq_words_uniref0.5_vsz30k_mfq2-vocab.txt"))

In [178]:
testX=[utils.convert_seq_ids(i, tokenizer, 200) for i in testseqs]

In [191]:
print(len(testX))
print(len(testX[i]), testX[i])

50
200 [2, 25764, 551, 8030, 382, 774, 4078, 1225, 19227, 221, 42, 10220, 18631, 5597, 23588, 487, 20658, 1694, 8985, 1761, 184, 42, 572, 21557, 12174, 97, 42, 294, 42, 27824, 13643, 42, 16760, 42, 307, 2963, 3094, 3197, 42, 809, 228, 174, 12695, 4237, 516, 329, 42, 29266, 42, 289, 3068, 42, 441, 2594, 7358, 147, 4053, 1039, 11092, 16108, 29399, 499, 3438, 6693, 40, 3977, 25178, 1614, 13758, 12766, 46, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [192]:
posscore=model.predict(testX)

In [193]:
prediction_scores = utils.func_predict_scores_dense1(testids, posscore, win=200, stride=80)
# prediction_scores = utils.func_predict_scores_bilstm(testids, posscore, win=200, stride=80)

1 >T0529_2_7-339 364-561_569
0.33941758 0.33326033 0.32708585 0.3359403 0.34338227 0.3282704 0.34528568 0.3380613 0.31502554 0.3308449 0.34833008 0.3262844 0.33076102 0.3343872 0.33344278 0.35347375 0.3398589 0.3305922 0.33284208 0.34428412 0.34029058 0.31070566 0.31645417 0.32821932 0.30764356 0.29314706 0.30974746 0.29356322 0.2733824 0.29224432 0.30116364 0.28958935 0.2907751 0.29118532 0.29823005 0.28640732 0.26886716 0.27726465 0.2813403 0.28315905 0.28629616 0.28759596 0.272971 0.27987558 0.28595114 0.28430197 0.28609544 0.28014687 0.28715238 0.28290862 0.26863098 0.28557938 0.26576975 0.27158803 0.28447437 0.26779652 0.273538 0.2807258 0.27449635 0.27778238 0.26806554 0.27639413 0.2725141 0.28870413 0.2802026 0.28381318 0.27757233 0.28715414 0.2732252 0.28846818 0.2750508 0.2875346 0.2611667 0.2737748 0.26240754 0.28065327 0.26757956 0.2686684 0.27181944 0.28736627 0.31152844 0.304778 0.30593958 0.30573988 0.31148666 0.3063308 0.31014872 0.3129944 0.29987454 0.30960298 0.3158399

In [ ]:
# print("Number of sequences:", len(prediction_scores.keys()))
# for k, v in prediction_scores.items():
#     print(k)
#     print(v)

In [194]:
(ids, true_labels) = utils.process_inputlabels(label_file_unprocessed)

In [197]:
threshold = 0.29
predicted_labels = utils.get_prediction_labels(prediction_scores.values(), threshold)

print(" ".join(seqfile.split("/")[3].split("_")[:2]))
print(len(predicted_labels), len(true_labels))

if len(predicted_labels) == len(true_labels):
    tp, tn, fp, fn = utils.get_tp_tn_fp_fn(predicted_labels, true_labels)
    print("tp, tn, fp, fn = ", tp, tn, fp, fn)
    print("--------------------------------")
    print("threshold=", threshold)
    print("--------------------------------")
    utils.get_results(tp, tn, fp, fn)
else:
    print("Error: Use right files")    

casp9 2domain
14 14
tp, tn, fp, fn =  637 1766 692 782
--------------------------------
threshold= 0.29
--------------------------------
Precision =  0.47930775018811134
Recall =  0.448907681465821
Accuracy =  0.6198091307712149


In [ ]:
if len(predicted_labels) == len(true_labels):
    for i in range(len(ids)):
        print(ids[i])
        print("Predict:", predicted_labels[i])
        print("True:   ", true_labels[i])
        # print(len(predict_labels[i]))
else:
    print("Error: Use right files")

>T0436_2_1-66 303-415 67-302
Predict: 11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111000001111111111111111110000000000000000000000000000000000000000000000000000000000001111111111111111111111111111111111110
True:    11111111111111111111100000000000000000000000011111111111111111111111111111111111111111100000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000111111111111111111111111111111111111111111000000000000000000000000000000000000000000000000000000000000000000000001111111111111111111111111
>T0447_3_1-115 235-427 116-234 428-517
Predict: 11111111111111111111111111111111111111111111111111111111111111111